In [ ]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import math
import numpy as np
import time
from scipy import signal
from sklearn.decomposition import NMF
plt.close('all')

#opens new window
%matplotlib tk

Format = pyaudio.paInt16
Channels = 1
Rate = 44100
Chunk = 1024*8
lines = []

#pyaudio class isntance
p = pyaudio.PyAudio()

#stream objects to get data from the microphone
stream = p.open(
    format = Format,
    channels = Channels,
    rate = Rate,
    input = True,
    output = True,
    frames_per_buffer = Chunk
)

#create matplot figure
fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(15,8))

#varible for plotting
x = np.linspace(0, Chunk, 16)
x_fft = np.linspace(0, Rate, 257)

#create a line object pulling random data - can be random because it updates in a loop so y value doesn't matter
for i in range(4):
    lines.append(axs[i][0].semilogx(x_fft, np.random.rand(257), '-', lw=2))
    lines.append(axs[i][1].plot(x, np.random.rand(16), '-', lw=2))

#basic formatting for the plot
for i in range(4):
    axs[i][1].set_title('LOUDNESS')
    axs[i][1].set_xlabel('sample')
    axs[i][1].set_ylabel('volume')
    axs[i][1].set_ylim(0,400)
    axs[i][1].set_xlim(0,Chunk)

    axs[i][0].set_title('FFT')
    axs[i][0].set_xlabel('Frequency')
    axs[i][0].set_ylabel('Amplitude')
    axs[i][0].set_xlim(0,Rate/2)
    axs[i][0].set_ylim(0,500)

plt.tight_layout()

q=0
while True:
    #recieve binary data
    data = stream.read(Chunk, exception_on_overflow=False)
    
    #convert to int, make an array
    data_int = np.frombuffer(data, dtype = np.int16) 
    
    #sets the variables using scipys spectrogram feature, f is frequency, t is time, sxx is strength
    f, t, Sxx = signal.spectrogram(data_int, Rate, nperseg=512, nfft=512, noverlap=0, mode='psd')

    #Fits NMF Model
    model = NMF(n_components=4, solver='mu', init='nndsvd', random_state=0, max_iter=50, alpha=3, l1_ratio=0.8)
    W = model.fit_transform(Sxx)
    H = model.components_
    
    i=0
    j=0
    while(j < 8):
        lines[j][0].set_ydata(W[:,i])
        j+=1
        lines[j][0].set_ydata(H[i])
        j+=1
        i+=1
    
    #update figure canvas
    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.pause(0.00005)
    
    #used for testing
#     y = np.matmul(W, H)
#     print(np.sum(Sxx) - np.sum(y))
#     print(len(Sxx),len(Sxx[0]))
    q+=1    

/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '
/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/nmf.py:212: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  UserWarning)
/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/nmf.py:1035: ConvergenceWarning: Maximum number of iteration 50 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


In [24]:
import numpy as np
from sklearn.decomposition import NMF

X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
print(len(X))
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_
y = np.matmul(W, H)

print(X)
print("----")
print(W)
print()
print(H)
print("----")
print(y)



6
[[1.  1. ]
 [2.  1. ]
 [3.  1.2]
 [4.  1. ]
 [5.  0.8]
 [6.  1. ]]
----
[[0.         0.46880684]
 [0.55699523 0.3894146 ]
 [1.00331638 0.41925352]
 [1.6733999  0.22926926]
 [2.34349311 0.03927954]
 [2.78981512 0.06911798]]

[[2.09783018 0.30560234]
 [2.13443044 2.13171694]]
----
[[1.00063558 0.99936347]
 [1.99965977 1.00034074]
 [2.99965485 1.20034566]
 [3.9998681  1.0001321 ]
 [5.00009002 0.79990984]
 [6.00008587 0.999914  ]]


In [ ]:
"""
tests similarity between original and produced spectrogram

"""
import pyaudio
import struct
import matplotlib.pyplot as plt
import math
import numpy as np
import time
from scipy import signal
from sklearn.decomposition import NMF
plt.close('all')

#opens new window
%matplotlib tk

Format = pyaudio.paInt16
Channels = 1
Rate = 44100
Chunk = 1024*8
lines = []

#pyaudio class isntance
p = pyaudio.PyAudio()

#stream objects to get data from the microphone
stream = p.open(
    format = Format,
    channels = Channels,
    rate = Rate,
    input = True,
    output = True,
    frames_per_buffer = Chunk
)

#create matplot figure
fig, (ax,ax2) = plt.subplots(2, figsize=(8,8))


#basic formatting for the plot
for i in range(4):
    ax.set_title('SPECTROGRAM')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
    ax.set_ylim(0,2000)
    
    ax2.set_title('SPECTROGRAM')
    ax2.set_ylabel('Frequency [Hz]')
    ax2.set_xlabel('Time [sec]')
    ax2.set_ylim(0,2000)

plt.tight_layout()

q=0
while True:
    #recieve binary data
    data = stream.read(Chunk, exception_on_overflow=False)
    
    #convert to int, make an array
    data_int = np.frombuffer(data, dtype = np.int16)
    
    #sets the variables using scipys spectrogram feature, f is frequency, t is time, sxx is strength
    f, t, Sxx = signal.spectrogram(data_int, Rate, nperseg=1024, nfft=1024, noverlap=0, mode='magnitude')

    #Fits NMF Model
    model = NMF(n_components=4, init='random', random_state=0, max_iter=50)
    W = model.fit_transform(Sxx)
    H = model.components_
    y = np.matmul(W, H)
    
    c = ax.pcolor(t, f, Sxx)
    c2 = ax2.pcolor(t, f, y)

#     if q == 1:
#         fig.colorbar(f)
    
    #update figure canvas
    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.pause(0.00005)
    
    #used for testing
#     print(np.sum(Sxx) - np.sum(y))
    q+=1    

In [ ]:
import sys
import argparse

import numpy as np
from sklearn.decomposition import NMF

from preprocessing import preprocess
from mu import *
from als import *
from anls_as import anls_as


def print_clusters(W, features, cluster_size):
    '''
    Print clusters found on W.
    
    Parameters:
        W: ndarray
            - a matrix containing clusters found by non-negative matrix factorization
        features: list
        - a list of features
        cluster_size: int
            - an integer specifying the number of features in a cluster
    Returns:
        None
    '''
    print('---------------------------------------------------------------------')
    print('Discovered clusters')
    print('')

    sorted_W = W[:, np.argsort(W.sum(axis = 0))]
    
    for i in range(np.size(W, 1)):
        cluster = []
        idx = (-sorted_W[:,i]).argsort()[:cluster_size]
        for j in range(np.size(idx)):
            cluster.append(features[idx[j]])
        print('Cluster ' + str(i + 1) + ': ' + ', '.join(cluster))

    print('')


def process_arg():
    '''
    Process the user input.
    Parameters:
        None
    Returns:
        args: argparse.Namespace()
    '''
    parser = argparse.ArgumentParser(description = "This program applies a nonnegative matrix factorization algorithms to a dataset for clustering.")
    parser.add_argument('-f', '--filename', type = str, required = True,  help = 'the input file name')
    parser.add_argument('-c', '--col_name', type = str, required = True,  help = 'the column of the input csv file for nonnegative matrix factorization.')
    parser.add_argument('-m', '--method', default = 'mu', choices = {'all', 'mu', 'als', 'anls_as', 'sklearn'}, type = str, required = True,  help = 'the NMF method to apply')
    parser.add_argument('-d', '--data_frac', default = 1, type = float, required = False, help = 'the amount of the data to be used')
    parser.add_argument('-r', '--random_sample', default = True, type = bool, required = False,  help = 'if set False, disables random sampling of the data')
    parser.add_argument('-n', '--num_max_feature', default = 1000, type = int, required = False,  help = 'the maximum number of features to be discovered in the dataset')
    parser.add_argument('-s', '--cluster_size', default = 10, type = int, required = False,  help = 'the number of features in each cluster')
    parser.add_argument('-k', '--num_clusters', default = 5, type = int, required = False,  help = 'the number of clusters to be discovered')
    parser.add_argument('-i', '--num_iters', default = 100, type = int, required = False,  help = 'the number of iterations to run a NMF algorithm')
    parser.add_argument('-p', '--print_enabled', default = False, type = bool, required = False,  help = 'if ture, output print statements')

    args = parser.parse_args()

    return args


def process_NMF(args, A, features):
    '''
    Using a specified algorithm by user, apply nonnegative matrix factorization to the given matrix A.
    Based on the factorization, print clusters.
    Paramters:
        args: Namespace
            - contains the arguments from the user
        A:
            - the matrix to factorize
        features:
            - discovered features of the input matrix
    Returns:
        None
    '''
    # Run a desired algorithm to perform non-negative matrix factorization on A
    if args.method == 'all':
        # Initialize W and H
        # Use the same matrices for all the algorithms for comparison
        init_W = np.random.rand(np.size(A, 0), args.num_clusters)
        init_H = np.random.rand(args.num_clusters, np.size(A, 1))

        # Run multiplicative updates with init_W and init_H
        W, H = mu(A, args.num_clusters, delta = 0.0000001, num_iter = args.num_iters, init_W = init_W, init_H = init_H, print_enabled = args.print_enabled)
        print_clusters(W, features, args.cluster_size)

        # Run alternating least squares with init_W and init_H
        W, H = als(A, args.num_clusters, num_iter = args.num_iters, init_W = init_W, init_H = init_H, print_enabled = args.print_enabled)
        print_clusters(W, features, args.cluster_size)

        # Run alternating non-negative least squares with active set with init_W and init_H
        W, H = anls_as(A, args.num_clusters, num_iter = args.num_iters, init_W = init_W, init_H = init_H, print_enabled = args.print_enabled)
        print_clusters(W, features, args.cluster_size)

    elif args.method == 'mu':
        W, H = mu(A, args.num_clusters, delta = 0.0000001, num_iter = args.num_iters, print_enabled = args.print_enabled)

    elif args.method == 'als':
        W, H = als(A, args.num_clusters, num_iter = args.num_iters, print_enabled = args.print_enabled)

    elif args.method == 'anls_as':
        W, H = anls_as(A, args.num_clusters, num_iter = args.num_iters, print_enabled = args.print_enabled)

    elif args.method == 'sklearn':
        model = NMF(n_components = args.num_clusters, init='nndsvd')
        W = model.fit_transform(A)
        H = model.components_

    # Print clusters found by non-negative matrix factorization
    if args.method != 'all':
        print_clusters(W, features, args.cluster_size)


if __name__ == '__main__':
    # Process arguments
    args = process_arg()

    # Preprocess the dataset
    try: 
        A, features = preprocess(args.filename, args.col_name, data_frac = args.data_frac, random_sample = args.random_sample, num_max_feature = args.num_max_feature, print_enabled = args.print_enabled)
    except FileNotFoundError:
        sys.exit("Error: File/Column not found")


/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '
/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '
